<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/download_sync_async.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilidades

## Classe que extrai nome e extensão de um arquivo em uma URL

Uma URL (Uniform Resource Locator), ou endereço web, é como um endereço postal na internet. Ela indica a localização exata de um recurso online, como uma página web, imagem, vídeo ou outro arquivo.

**Componentes de uma URL:**

  * **Esquema:** Indica o protocolo utilizado para acessar o recurso. Os mais comuns são:
      * **http:** Protocolo de transferência de hipertexto, utilizado para páginas web não seguras.
      * **https:** Versão segura do HTTP, que utiliza criptografia para proteger a comunicação.
      * **ftp:** Protocolo de transferência de arquivos, utilizado para transferir arquivos entre computadores.
      * **mailto:** Utilizado para enviar e-mails.
  * **Host:** Identifica o servidor onde o recurso está localizado. É composto pelo nome de domínio (ex: [URL inválido removido]) e, opcionalmente, o número da porta.
  * **Caminho:** Especifica a localização exata do recurso dentro do servidor. É semelhante à estrutura de pastas em um computador.
  * **Parâmetros de consulta:** Fornecem informações adicionais ao servidor, como resultados de busca ou configurações personalizadas. São separados do caminho por um ponto de interrogação (?) e os pares chave-valor são separados por um & (ampersand).
  * **Fragmento:** Indica uma parte específica de uma página, como uma seção ou um link dentro da página. É separado do resto da URL por um jogo da velha (\#).

**Exemplo:**

```
https://www.exemplo.com/pasta/arquivo.html?parametro1=valor1&parametro2=valor2#secao
```

  * **https:** Protocolo seguro.
  * **[www.exemplo.com](https://www.google.com/url?sa=E&source=gmail&q=https://www.exemplo.com):** Nome de domínio do servidor.
  * **/pasta/arquivo.html:** Caminho do arquivo dentro do servidor.
  * **?parametro1=valor1\&parametro2=valor2:** Parâmetros de consulta.
  * **\#secao:** Fragmento, indicando uma seção específica da página.

**Para que serve cada componente:**

  * **Esquema:** Define como o navegador deve se conectar ao servidor.
  * **Host:** Identifica o servidor específico onde o recurso está hospedado.
  * **Caminho:** Direciona o navegador para o local exato do arquivo dentro do servidor.
  * **Parâmetros de consulta:** Permitem que você passe informações adicionais ao servidor, como resultados de busca, configurações personalizadas ou dados de formulários.
  * **Fragmento:** Permite que você navegue diretamente para uma parte específica de uma página, sem precisar carregar a página inteira novamente.

In [22]:
from urllib.parse import urlparse
import os

class Util:
  def extrair_nome_extensao_url(self, url):
    try:
      parsed_url = urlparse(url)
      if parsed_url.scheme not in ('http', 'https', 'ftp'):
        raise ValueError(f"Unsupported protocol: {parsed_url.scheme}")

      caminho_arquivo = parsed_url.path
      if not caminho_arquivo:
        raise ValueError("Missing file path in URL")

      #nome_arquivo, extensao = os.path.splitext(os.path.basename(caminho_arquivo))
      nome_arquivo, extensao = os.path.basename(caminho_arquivo).rsplit('.', 1)

      if not nome_arquivo:
        raise ValueError("Missing file name")

      return nome_arquivo, extensao

    except Exception as ex:
      raise ValueError(f"{str(ex)}") from ex

In [27]:
util = Util()
url = "https://exemplo.com/caminho/para/arquivo.txt"
nome_arquivo, extensao = util.extrair_nome_extensao_url(url)
print(nome_arquivo)  # Saída: arquivo
print(extensao)      # Saída: .txt

arquivo
txt


In [28]:
url2 = "https://www.exemplo.com/pasta/arquivo.html?parametro1=valor1&parametro2=valor2#secao"
nome_arquivo, extensao = util.extrair_nome_extensao_url(url2)
print(nome_arquivo)  # Saída: arquivo
print(extensao)      # Saída: .html

arquivo
html


## Limpa a estrutura das pastas que vão armazenar os arquivos

In [29]:
!rm -rf sincrono && mkdir sincrono
!rm -rf assincrono && mkdir assincrono
!rm -rf threads && mkdir threads

In [25]:
!ls -lia

total 28
6815750 drwxr-xr-x 1 root root 4096 Jan 13 15:19 .
5505040 drwxr-xr-x 1 root root 4096 Jan 13 14:25 ..
5509883 drwxr-xr-x 2 root root 4096 Jan 13 15:19 assincrono
2621447 drwxr-xr-x 4 root root 4096 Jan  9 14:24 .config
6815751 drwxr-xr-x 1 root root 4096 Jan  9 14:24 sample_data
5509882 drwxr-xr-x 2 root root 4096 Jan 13 15:19 sincrono
5509884 drwxr-xr-x 2 root root 4096 Jan 13 15:19 threads


In [26]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


## Prepara as URLs dos arquivos de exemplo (Download de arquivos públicos)

In [30]:
my_raw_data_site = "https://raw.githubusercontent.com/armandossrecife/teste/main"
my_url1 = my_raw_data_site + "/" + "Adrienne.mp4"
my_url2 = my_raw_data_site + "/" + "Pizigani_1367_Chart_10MB.jpg"
my_url3 = my_raw_data_site + "/" + "Kalimba.mp3"
my_url4 = my_raw_data_site + "/" + "screen_matrix.jpeg"
my_url5 = my_raw_data_site + "/" + "demo.zip"

my_urls = [my_url1, my_url2, my_url3, my_url4, my_url5]

my_util = Util()
my_filenames = []
for url in my_urls:
  print(url)
  nome_arquivo, extensao = my_util.extrair_nome_extensao_url(url)
  filename = f"{nome_arquivo}.{extensao}"
  print(f"{nome_arquivo}.{extensao}")
  my_filenames.append(filename)

https://raw.githubusercontent.com/armandossrecife/teste/main/Adrienne.mp4
Adrienne.mp4
https://raw.githubusercontent.com/armandossrecife/teste/main/Pizigani_1367_Chart_10MB.jpg
Pizigani_1367_Chart_10MB.jpg
https://raw.githubusercontent.com/armandossrecife/teste/main/Kalimba.mp3
Kalimba.mp3
https://raw.githubusercontent.com/armandossrecife/teste/main/screen_matrix.jpeg
screen_matrix.jpeg
https://raw.githubusercontent.com/armandossrecife/teste/main/demo.zip
demo.zip


# Download síncrono

https://requests.readthedocs.io

Requests is an elegant and simple HTTP library for Python, built for human beings.

Exemplo de uso da biblioteca requets:

In [35]:
import requests

def buscar_cep(cep):
    url = f'https://viacep.com.br/ws/{cep}/json/'
    response = requests.get(url)
    dados = response.json()
    return dados

cep = '64007250'
resultado = buscar_cep(cep)
print(resultado)
print(f"Logradouro: {resultado['logradouro']}")
print(f"Bairro: {resultado['bairro']}")
print(f"Cidade: {resultado['localidade']}")
print(f"Estado: {resultado['uf']}")

{'cep': '64007-250', 'logradouro': 'Rua Território Fernando de Noronha', 'complemento': '', 'unidade': '', 'bairro': 'Aeroporto', 'localidade': 'Teresina', 'uf': 'PI', 'estado': 'Piauí', 'regiao': 'Nordeste', 'ibge': '2211001', 'gia': '', 'ddd': '86', 'siafi': '1219'}
Logradouro: Rua Território Fernando de Noronha
Bairro: Aeroporto
Cidade: Teresina
Estado: PI


## Funções para download de arquivos

In [37]:
import requests

# Faz o download de um único arquivo
def download_one_file(url, filename, path):
  response = requests.get(url, stream=True)
  if response.status_code == 200:
    total_size = int(response.headers['content-length'])  # Get total file size
    print(f"Total file size: {total_size} bytes")
    path = os.path.join(path, filename)
    with open(path, 'wb') as f:
      for chunk in response.iter_content(1024):
        f.write(chunk)
    print(f"Downloaded {filename}")
  else:
    print(f"Failed to download {filename}")

# Faz o download sincrono de uma lista de arquivos
def download_files_synchronous(my_urls, my_filenames, path):
  # Download each file synchronously
  for url, filename in zip(my_urls, my_filenames):
    download_one_file(url, filename, path)

In [39]:
my_urls

['https://raw.githubusercontent.com/armandossrecife/teste/main/Adrienne.mp4',
 'https://raw.githubusercontent.com/armandossrecife/teste/main/Pizigani_1367_Chart_10MB.jpg',
 'https://raw.githubusercontent.com/armandossrecife/teste/main/Kalimba.mp3',
 'https://raw.githubusercontent.com/armandossrecife/teste/main/screen_matrix.jpeg',
 'https://raw.githubusercontent.com/armandossrecife/teste/main/demo.zip']

In [6]:
import datetime

now1 = datetime.datetime.now()
print(now1)

download_files_synchronous(my_urls, my_filenames, 'sincrono')
print("All files downloaded (synchronously)")

now2 = datetime.datetime.now()
print(now2)

time_diff = now2 - now1
print(time_diff)

2025-01-13 14:26:12.107071
Total file size: 14944332 bytes
Downloaded Adrienne.mp4
Total file size: 10174706 bytes
Downloaded Pizigani_1367_Chart_10MB.jpg
Total file size: 8414449 bytes
Downloaded Kalimba.mp3
Total file size: 265136 bytes
Downloaded screen_matrix.jpeg
Total file size: 69856 bytes
Downloaded demo.zip
All files downloaded (synchronously)
2025-01-13 14:26:14.794788
0:00:02.687717


# Download assíncrono

https://docs.aiohttp.org

Asynchronous HTTP Client/Server for asyncio and Python.

https://en.wikipedia.org/wiki/Asynchrony_(computer_programming)

https://en.wikipedia.org/wiki/Async/await

A biblioteca **aiohttp** é uma ferramenta fundamental para o desenvolvimento de aplicações web assíncronas em Python. Ela permite criar tanto clientes HTTP (para fazer requisições a outros servidores) quanto servidores HTTP (para criar suas próprias APIs ou serviços web).

**Por que usar aiohttp?**

* **Assincronia:**  A principal vantagem do aiohttp é sua capacidade de lidar com múltiplas requisições de forma não bloqueante. Isso significa que seu código pode continuar processando outras tarefas enquanto aguarda uma resposta de uma requisição HTTP, tornando suas aplicações mais eficientes e escaláveis.
* **Desempenho:** Ao utilizar a biblioteca asyncio do Python, o aiohttp oferece um desempenho superior em comparação com bibliotecas síncronas como requests, especialmente em cenários com alto volume de requisições.
* **Facilidade de uso:** A API do aiohttp é intuitiva e bem documentada, tornando-a fácil de aprender e usar.
* **Recursos completos:** A biblioteca oferece uma ampla gama de recursos, incluindo suporte a WebSocket, middleware, roteamento e muito mais.

**Como funciona:**

* **Cliente HTTP:** Para fazer requisições HTTP, você cria um cliente aiohttp e utiliza métodos como `get()`, `post()`, `put()` e `delete()`. As respostas são retornadas como objetos assíncronos, permitindo que você continue processando outras tarefas enquanto aguarda a resposta.
* **Servidor HTTP:** Para criar um servidor HTTP, você define rotas e handlers para tratar as diferentes requisições. O aiohttp se encarrega de gerenciar as conexões e as tarefas de forma assíncrona.

**Exemplo básico:**

```python
import asyncio
import aiohttp

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'https://www.example.com')
        print(html)

asyncio.run(main())
```

**Conceitos chave:**

* **asyncio:** A biblioteca asyncio é a base do aiohttp. Ela fornece a infraestrutura para programação assíncrona em Python.
* **Coroutines:** Coroutines são funções que podem suspender e retomar sua execução. O aiohttp utiliza coroutines para lidar com requisições e respostas de forma assíncrona.
* **Tasks:** Tasks são coroutines agendadas para execução em um loop de eventos.
* **WebSockets:** O aiohttp oferece suporte nativo a WebSockets, permitindo a comunicação bidirecional em tempo real entre o cliente e o servidor.

**Quando usar aiohttp:**

* **Aplicações web de alta performance:** Se você precisa lidar com um grande volume de requisições simultâneas.
* **APIs REST:** Para criar APIs RESTful escaláveis e eficientes.
* **Web Scraping:** Para extrair dados de sites de forma rápida e eficiente.
* **Aplicações de chat e jogos online:** Para implementar a comunicação em tempo real.

In [7]:
!pip install aiohttp

In [8]:
!pip install aiodns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 5.3 MB/s eta 0:00:00


Exemplo básico:

In [47]:
import asyncio
import aiohttp

# Faz a requisição e aguarda o conteúdo da resposta
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

# Cria a sessão de um cliente para recuperar os dados da requisição
async def main_basic():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'https://www.google.com')
        print(html)

# Função auxiliar para encapsular uma chamada assíncrona dentro do Google Colabs
async def call_teste_async():
  await main_basic()

# Faz a chamada sob demanda
await call_teste_async()

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"><meta content="noodp, " name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="4zIdoQSmU7cZK16fa-Po6w">(function(){var _g={kEI:'bzaFZ_OgD8-SwbkPmtCDkQI',kEXPI:'0,3700257,1127,538661,2872,2891,8349,34679,30022,360901,45786,9779,99404,3801,2412,58603,27535,11813,1633,29278,15664,11419,5213672,766,5992089,2842796,27977918,25224045,4636,16436,106667,885,14280,8182,5933,8939,13974,20583,13629,5382,2659,3435,3319,23879,9139,4599,328,6225,10418,12989,6,10211,686,16265,3069,6973,3548,1341,5328,8379,50,8161,873,1,2,2410,4134,7265,10550,3491,33,4669,4372,1175,16492,

In [48]:
import asyncio
import aiohttp
import datetime

# Função integrada (chamada da sessão e requisção integradas)
async def teste_async():
  async with aiohttp.ClientSession() as session:
    async with session.get('http://python.org') as response:
      print("Status:", response.status)
      print("Content-type:", response.headers['content-type'])
      html = await response.text()

async def call_teste_async():
  await teste_async()

await call_teste_async()

Status: 200
Content-type: text/html; charset=utf-8


## Função que faz o download de forma assícrona

In [50]:
async def download_async(url, filename, path):
  """Downloads a file from the given URL and saves it with the specified filename."""
  async with aiohttp.ClientSession() as session:
    async with session.get(url) as response:
      total_size = int(response.headers['content-length'])  # Get total file size
      print(f"Total file size: {total_size} bytes")
      if response.status == 200:
        path = os.path.join(path, filename)
        with open(path, 'wb') as f:
          # Read content in chunks using aiohttp.read()
          while True:
            chunk = await response.content.read(1024)  # Read in chunks of 1024 bytes
            if not chunk:
              break
            f.write(chunk)
        print(f"Downloaded {filename}")
      else:
        print(f"Failed to download {filename}")

# Faz o download de uma lista de arquivos (my_urls)
async def download_files_asynchronous(my_urls, my_filenames, path):
  """Downloads all files asynchronously."""
  tasks = [] # Lista de tarefas para guardar cada cada requisição em uma sessão
  for url, filename in zip(my_urls, my_filenames):
    tasks.append(asyncio.create_task(download_async(url, filename, path)))

  # Run all tasks concurrently using asyncio.run
  await asyncio.gather(*tasks)

In [51]:
now1 = datetime.datetime.now()
print(now1)

async def main():
  await download_files_asynchronous(my_urls, my_filenames, 'assincrono')

# Call main as an async function and await it
await main()

now2 = datetime.datetime.now()
print(now2)

time_diff = now2 - now1
print(time_diff)

2025-01-13 15:57:29.425827
Total file size: 69856 bytes
Total file size: 265136 bytes


/usr/local/lib/python3.10/dist-packages/pycares/__init__.py:819: RuntimeWarning: coroutine 'main' was never awaited
  if _ffi.NULL != _lib.ares_inet_ntop(s.sin6_family, _ffi.addressof(s.sin6_addr), ip, _lib.INET6_ADDRSTRLEN):


Downloaded demo.zip
Downloaded screen_matrix.jpeg
Total file size: 14944332 bytes
Total file size: 10174706 bytes
Total file size: 8414449 bytes
Downloaded Pizigani_1367_Chart_10MB.jpg
Downloaded Adrienne.mp4
Downloaded Kalimba.mp3
2025-01-13 15:57:29.974725
0:00:00.548898


# Downloads usando Threads

https://docs.python.org/3/library/threading.html

https://en.wikipedia.org/wiki/Thread_(computing)

## Threads: Os Fios da Execução

**O que é uma Thread?**

Em termos simples, uma **thread** é como um fio dentro de um programa de computador. Esse fio representa uma sequência de instruções que podem ser executadas de forma independente, mas que compartilham os mesmos recursos de um processo. Imagine um programa como uma fábrica: as threads são como as linhas de montagem, cada uma realizando uma tarefa específica, mas todas trabalhando dentro da mesma fábrica (o processo).

**Para que servem as Threads?**

* **Concorrência:** Permitem que um programa execute várias tarefas de forma simultânea ou quase simultânea, melhorando o desempenho.
* **Responsividade:** Evitam que a interface de um programa "congele" enquanto uma tarefa demorada é realizada, permitindo que o usuário continue interagindo com o programa.
* **Aproveitamento de múltiplos núcleos:** Em processadores com múltiplos núcleos, as threads podem ser distribuídas por esses núcleos, aumentando a eficiência.

**Como as Threads funcionam?**

* **Compartilhamento de recursos:** Todas as threads de um mesmo processo compartilham a mesma memória, as mesmas variáveis e os mesmos arquivos.
* **Execução independente:** Cada thread possui seu próprio contador de programa, pilha e registros, o que permite que elas executem instruções diferentes e em ordem diferente.
* **Gerenciamento pelo sistema operacional:** O sistema operacional alterna a execução entre as threads, dando a cada uma um pequeno período de tempo para executar suas instruções.

**Exemplo:**

Imagine um navegador web. Cada aba aberta pode ser considerada uma thread. Enquanto você navega em uma aba, outras abas podem estar carregando conteúdo em segundo plano. Isso é possível porque cada aba é uma thread independente, que pode ser executada simultaneamente.

**Em resumo:**

As threads são uma ferramenta fundamental na programação moderna, permitindo que os programas sejam mais responsivos, eficientes e capazes de realizar múltiplas tarefas simultaneamente. Ao entender o conceito de threads, você pode escrever programas mais complexos e otimizados.

In [52]:
import threading
import requests

def download_files_via_threads(urls, filenames, path):
  """Downloads multiple files asynchronously using threads."""
  threads = []
  for url, filename in zip(urls, filenames):
    thread = threading.Thread(target=download_one_file(url, filename, path), args=(url, filename, path))
    threads.append(thread)
    thread.start()

  # Wait for all threads to finish (blocking)
  for thread in threads:
    thread.join()

In [53]:
now1 = datetime.datetime.now()
print(now1)

download_files_via_threads(my_urls, my_filenames, 'threads')

now2 = datetime.datetime.now()
print(now2)

time_diff = now2 - now1
print(time_diff)

2025-01-13 16:02:11.353324
Total file size: 14944332 bytes
Downloaded Adrienne.mp4
Total file size: 10174706 bytes
Downloaded Pizigani_1367_Chart_10MB.jpg
Total file size: 8414449 bytes
Downloaded Kalimba.mp3
Total file size: 265136 bytes
Downloaded screen_matrix.jpeg
Total file size: 69856 bytes
Downloaded demo.zip
2025-01-13 16:02:12.200063
0:00:00.846739


# Conceitos chaves

## Chamadas síncronas

In [55]:
import datetime
import time


def task1_sync():
    """Simulates a long-running task that takes 2 seconds."""
    print("Task 1 started")
    print(datetime.datetime.now())
    time.sleep(2)  # Simulate work for 2 seconds
    print("Task 1 finished")
    print(datetime.datetime.now())


def task2_sync():
    """Simulates a shorter task that takes 1 second."""
    print("Task 2 started")
    print(datetime.datetime.now())
    time.sleep(1)  # Simulate work for 1 second
    print("Task 2 finished")
    print(datetime.datetime.now())


def task3_sync(duration=3):
    """Simulates a task that takes a specified duration (in seconds)."""
    print(f"Task 3 started (duration: {duration} seconds)")
    print(datetime.datetime.now())
    time.sleep(duration)  # Simulate work for the specified duration
    print("Task 3 finished")
    print(datetime.datetime.now())


def run_tasks_sync():
    """Runs three tasks sequentially using synchronous calls."""
    task1_sync()
    task2_sync()
    task3_sync()  # Call the new task3_sync function

    print("All tasks finished")

In [56]:
print("#"*50)
print("Chamada síncrona")
now1 = datetime.datetime.now()
print(now1)
print("-"*50)

run_tasks_sync()

now2 = datetime.datetime.now()
print(now2)
print("-"*50)
print(f"Tempo total: {now2-now1}")

##################################################
Chamada síncrona
2025-01-13 16:04:45.469641
--------------------------------------------------
Task 1 started
2025-01-13 16:04:45.471416
Task 1 finished
2025-01-13 16:04:47.474088
Task 2 started
2025-01-13 16:04:47.474160
Task 2 finished
2025-01-13 16:04:48.475422
Task 3 started (duration: 3 seconds)
2025-01-13 16:04:48.475503
Task 3 finished
2025-01-13 16:04:51.478761
All tasks finished
2025-01-13 16:04:51.479011
--------------------------------------------------
Tempo total: 0:00:06.009370


## Chamadas assíncronas

In [60]:
import asyncio
import datetime

async def task1_async():
    """Simulates a long-running task that takes 2 seconds."""
    print("Task 1 started")
    print(datetime.datetime.now())
    await asyncio.sleep(2)  # Simulate work for 2 seconds
    print("Task 1 finished")
    print(datetime.datetime.now())

async def task2_async():
    """Simulates a shorter task that takes 1 second."""
    print("Task 2 started")
    print(datetime.datetime.now())
    await asyncio.sleep(1)  # Simulate work for 1 second
    print("Task 2 finished")
    print(datetime.datetime.now())

async def task3_async(duration=3):
    """Simulates a task that takes a specified duration (in seconds)."""
    print(f"Task 3 started (duration: {duration} seconds)")
    print(datetime.datetime.now())
    await asyncio.sleep(duration)  # Simulate work for the specified duration
    print("Task 3 finished")
    print(datetime.datetime.now())

async def run_tasks_async():
    """Runs three tasks concurrently using asyncio."""
    task1_future = asyncio.create_task(task1_async())
    task2_future = asyncio.create_task(task2_async())
    task3_future = asyncio.create_task(task3_async())  # Add task3_async coroutine

    # Wait for all tasks to complete concurrently (non-blocking)
    await task1_future
    await task2_future
    await task3_future

    print("All tasks finished")

In [58]:
async def call_run_tasks():
  await run_tasks_async()

print("#"*50)
print("Chamada assíncrona")
now1 = datetime.datetime.now()
print(now1)
print("-"*50)

await call_run_tasks()

now2 = datetime.datetime.now()
print(now2)
print("-"*50)
print(f"Tempo total: {now2-now1}")

##################################################
Chamada assíncrona
2025-01-13 16:06:53.418277
--------------------------------------------------
Task 1 started
2025-01-13 16:06:53.418960
Task 2 started
2025-01-13 16:06:53.421841
Task 3 started (duration: 3 seconds)
2025-01-13 16:06:53.421933
Task 2 finished
2025-01-13 16:06:54.423333
Task 1 finished
2025-01-13 16:06:55.423701
Task 3 finished
2025-01-13 16:06:56.423316
All tasks finished
2025-01-13 16:06:56.423602
--------------------------------------------------
Tempo total: 0:00:03.005325


# Usando threads

In [61]:
import threading
import time

def task1():
    """Simulates a long-running task (2 seconds) using threading."""
    print("Task 1 started (Thread)")
    print(datetime.datetime.now())
    time.sleep(2)  # Simulate work for 2 seconds
    print("Task 1 finished (Thread)")
    print(datetime.datetime.now())

def task2():
    """Simulates a shorter task (1 second) using threading."""
    print("Task 2 started (Thread)")
    print(datetime.datetime.now())
    time.sleep(1)  # Simulate work for 1 second
    print("Task 2 finished (Thread)")
    print(datetime.datetime.now())

def task3(duration=3):
    """Simulates a task with a specified duration (in seconds) using threading."""
    print(f"Task 3 started (Thread, duration: {duration} seconds)")
    print(datetime.datetime.now())
    time.sleep(duration)  # Simulate work for the specified duration
    print("Task 3 finished (Thread)")
    print(datetime.datetime.now())

def run_tasks_via_threads():
    """Runs three tasks concurrently using threads."""
    thread1 = threading.Thread(target=task1)
    thread2 = threading.Thread(target=task2)
    thread3 = threading.Thread(target=task3)  # Create a thread for task3

    # Start threads
    thread1.start()
    thread2.start()
    thread3.start()

    # Wait for all threads to finish
    thread1.join()
    thread2.join()
    thread3.join()

    print("All tasks finished (Threads)")

In [62]:
print("#"*50)
print("Chamada assíncrona usando Threads")
now1 = datetime.datetime.now()
print(now1)
print("-"*50)

run_tasks_via_threads()

now2 = datetime.datetime.now()
print(now2)
print("-"*50)
print(f"Tempo total: {now2-now1}")

##################################################
Chamada assíncrona usando Threads
2025-01-13 16:10:22.573037
--------------------------------------------------
Task 1 started (Thread)
2025-01-13 16:10:22.573744
Task 2 started (Thread)
2025-01-13 16:10:22.580002
Task 3 started (Thread, duration: 3 seconds)
2025-01-13 16:10:22.581141
Task 2 finished (Thread)
2025-01-13 16:10:23.581742
Task 1 finished (Thread)
2025-01-13 16:10:24.576014
Task 3 finished (Thread)
2025-01-13 16:10:25.584413
All tasks finished (Threads)
2025-01-13 16:10:25.586012
--------------------------------------------------
Tempo total: 0:00:03.012975
